In [11]:
import mysql.connector

print("Creating Connection . . .")
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="aingg",
    database="35mmc"
)
curr = db.cursor()
print("Connceted")

# Getting all rows
print("Getting Data . . .")
curr.execute("SELECT * FROM 35mmc_raw_bk")

# fetch all
rawResult = curr.fetchall()
# close connection
print("Closing Connection . . .")
db.close()
print("Disconnected")


import re
import spacy
from nltk.corpus import stopwords

class TextCleaner:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.stop_words = set(stopwords.words('english'))

    def normalize(self, raw_text):
        """Remove special characters and lowercase text"""
        return re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", raw_text.lower())

    def remove_stopwords(self, raw_text):
        """Remove stopwords"""
        words = [word for word in raw_text.split() if word not in self.stop_words]
        return " ".join(words)

    def lemmatize(self, raw_text):
        """Perform lemmatization"""
        doc = self.nlp(raw_text)
        return [token.lemma_ for token in doc]

    def clean(self, raw_text):
        """Clean text by normalizing, removing stopwords, and lemmatizing"""
        raw_text = self.normalize(raw_text)
        raw_text = self.remove_stopwords(raw_text)
        return self.lemmatize(raw_text)
    

    


from collections import Counter
from tqdm import tqdm
import pandas as pd
import time
# from concurrent.futures import ThreadPoolExecutor

start = time.time()


invertedIndex = pd.DataFrame({'Gram':[], 'DocsFreq':[], 'DocsID_Dict':[]})
invertedIndex['DocsFreq'] = invertedIndex['DocsFreq'].astype(int)

# for i in tqdm(range(3)):
for i in tqdm(range(len(rawResult))):
    
    # get cleaned token
    text_cleaner = TextCleaner()
    normalized_text = text_cleaner.normalize(rawResult[i][2])
    no_stopwords_text = text_cleaner.remove_stopwords(normalized_text)
    lemmatized_text = text_cleaner.lemmatize(no_stopwords_text)
    
    # get token frequency
    freq_dict = Counter(lemmatized_text)
    
    for word in freq_dict:
        # If token already exists in dataframe
        if len(invertedIndex.loc[invertedIndex['Gram'] == word]) == 1:
            invertedIndex.loc[invertedIndex['Gram'] == word]['DocsID_Dict'][0][rawResult[i][3]] = freq_dict[word]
            invertedIndex.loc[invertedIndex['Gram'] == word, 'DocsFreq'] += 1
        
        # New Words
        else:
            test_gram = word
            test_DocsFreq = 1
            test_DocsID_Dict = {
                rawResult[i][3] : freq_dict[word]
            }
            new_row = pd.DataFrame({'Gram':[test_gram], 'DocsFreq':[test_DocsFreq], 'DocsID_Dict':[test_DocsID_Dict]})
            invertedIndex = pd.concat([invertedIndex, new_row])

end = time.time()        

print("Runtime : ", end-start, " sec.")

Creating Connection . . .
Connceted
Getting Data . . .
Closing Connection . . .
Disconnected


100%|██████████| 1093/1093 [56:36<00:00,  3.11s/it] 

Runtime :  3396.4911439418793  sec.


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
invertedIndex

In [14]:
from pathlib import Path  

# filepath = Path('webData.csv')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# webData.to_csv(filepath)  

filepath = Path('invertedIndex.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
invertedIndex.to_csv(filepath)  

In [15]:
(end - start)/60

56.60818573236465